<center><h1>&#128121 Primer trabajo: Métodos de Machine Learning no supervisado &#128128</h1></center>
<center><h3>Física Computacional II</h3></center>
<p><center><i>Sebastián Marín Ruiz </i></center></p>
<p><center><i>Facultad de Ciencias Básicas y Tecnologías </i></center></p>
<p><center><i>Universidad del Quindío</i></center></p>

In [9]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as clrs
import os
import random
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

In [2]:
# Ruta al directorio que contiene las imágenes
ruta_directorio = 'JEPG'

# Inicializa la variable para la menor dimensión encontrada
menor_dimension = float('inf')

# Itera a través de todas las imágenes en el directorio
for nombre_archivo in os.listdir(ruta_directorio):
    # Lee la imagen actual
    imagen = cv2.imread(os.path.join(ruta_directorio, nombre_archivo))
    
    # Obtiene las dimensiones de la imagen actual
    altura, ancho, canales = imagen.shape
    
    # Actualiza la variable para la menor dimensión encontrada, si corresponde
    if altura * ancho < menor_dimension:
        menor_dimension = altura * ancho
        imagen_menor_dimension = imagen
        dimensiones_imagen_menor_dimension = (ancho, altura)

# Muestra la imagen con la menor dimensión encontrada
#cv2.imshow('Imagen con menor dimensión', imagen_menor_dimension)
#cv2.waitKey(0)
#cv2.destroyAllWindows()


In [3]:
dimensiones_imagen_menor_dimension

(360, 444)

<h3> Reescalación de imágenes </h3>
<hr>

In [3]:
nuevo_ancho = 256
nueva_altura = 256
img_reescalada = "IMG_recortadas"

for i in os.listdir(ruta_directorio):
    imagen = cv2.imread(os.path.join(ruta_directorio, i))
    imagen_recortada = cv2.resize(imagen, (nuevo_ancho, nueva_altura))
    
    cv2.imwrite(os.path.join(img_reescalada, i), imagen_recortada)

<h3>Reducción de dimensionalidad</h3>
<hr>

In [4]:
# Especifica la ruta de la carpeta que contiene las imágenes
ruta_img = "IMG_recortadas"

# Selecciona 1000 imágenes aleatorias
file_names = os.listdir(ruta_img)
indices_aleatorios = random.sample(range(len(file_names)), 1000)

images = []
for i in indices_aleatorios:
    file_name = file_names[i]
    if file_name.endswith('.jpg'):
        image = cv2.imread(os.path.join(ruta_img, file_name))
        images.append(image)

np.shape(images)

(1000, 256, 256, 3)

In [5]:
# Convierte las imágenes a un array NumPy
images = np.array(images)
images_gray = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]
images_gray = np.array(images_gray)
flat_images = images_gray.reshape((len(images_gray), -1))
print(np.shape(flat_images))

tsne = TSNE(n_components = 2, random_state = 0)
X_tsne = tsne.fit_transform(flat_images)

(1000, 65536)


C:\Users\s\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
C:\Users\s\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [6]:
# Visualiza los resultados con OpenCV
max_x, max_y = np.max(X_tsne, axis=0)
min_x, min_y = np.min(X_tsne, axis=0)
range_x = max_x - min_x
range_y = max_y - min_y
scale_factor = 500 / max(range_x, range_y)

In [7]:
color_map = cm.get_cmap('jet')
color_norm = clrs.Normalize(vmin=0, vmax=255)  # normalizador de 0 a 255

color_circulo = cm.ScalarMappable(norm=color_norm, cmap=color_map)

In [8]:
img_output = np.ones((550, 550, 3), dtype=np.uint8) * 255
for i in range(X_tsne.shape[0]):
    x = int((X_tsne[i, 0] - min_x) * scale_factor) + 25
    y = int((X_tsne[i, 1] - min_y) * scale_factor) + 25
    color = tuple(map(int, 255 * color_circulo.to_rgba(flat_images[i])[0, :3]))
    img_output = cv2.circle(img_output, (x, y), 5, color, -1)

cv2.imshow('T-SNE', img_output)
cv2.waitKey(0)
cv2.destroyAllWindows()